<a href="https://colab.research.google.com/github/serialize02/2023final/blob/main/regression_ipynb_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading a dataset

In [1]:
import pandas as pd

### computer hardware dataset

classification 에서 다양한 입력이 들어가서 하나의 label 이 출력된 것처럼,
regression에서도 숫자 여러개가 들어가면 하나의 숫자가 예측됨

우리가 예측해낼 부분은 erp (estimated relative(?) performance)
이런 값이 있을 때 얼마나 performance 가 가능한가

정제작업: regression은 무조건 numerical data 가 들어가야 하기 때문에 vendor 이름을 그대로 사용할 수 없음 -> 다 숫자로 바꿔줘라,, 모델명도

In [2]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
computer_hardware = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data', header=None, names=['vendor', 'model', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'label'])
computer_hardware['vendor'] = le.fit_transform(computer_hardware['vendor'])
computer_hardware['model'] = le.fit_transform(computer_hardware['model'])
computer_hardware.head()

,vendor,model,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,label
0,0,29,125,256,6000,256,16,128,198,199
1,1,62,29,8000,32000,32,8,32,269,253
2,1,63,29,8000,32000,32,8,32,220,253
3,1,64,29,8000,32000,32,8,32,172,253
4,1,65,29,8000,16000,32,8,16,132,132


### parkinsons dataset

마지막거를 label 로 통합해서 바꿔주자

In [3]:
parkinsons = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/telemonitoring/parkinsons_updrs.data')
parkinsons.rename(columns={'motor_UPDRS':'label'}, inplace=True)
parkinsons.head()

,subject#,age,sex,test_time,label,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


## Comparing Models

In [4]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

models = [
    ('LR', LinearRegression()),
    ('L', Lasso()),
    ('R', Ridge()),
    ('SVR', LinearSVR()),
    ('DT', DecisionTreeRegressor()),
    ('NN', MLPRegressor()),
]

마지막 뉴럴넷만 알고 있으면 됨

### test a single model (e.g. Linear Regression)

In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

dataset_name = 'computer_hardware'
dataset = computer_hardware

X = np.array(dataset.drop(['label'], axis=1))
y = np.array(dataset['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

name ='LR'
model = LinearRegression()
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(dataset_name, name, accuracy)

computer_hardware LR 0.9438611618299353


In [6]:
model.predict(np.array([22,   122,    40,  8000, 16000,    32,     8,    16, 214]).reshape(1,9))

array([181.53279469])

### Evaluate all the models

In [7]:
for dataset_name, dataset in [('computer_hardware', computer_hardware), ('parkinsons', parkinsons)]:
    X = np.array(dataset.drop(['label'], axis=1))
    y = np.array(dataset['label'])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for name, model in models:
        model.fit(X_train, y_train)
        accuracy = model.score(X_test, y_test)
        print(dataset_name, name, accuracy)

computer_hardware LR 0.9438611618299353
computer_hardware L 0.9439215152431056
computer_hardware R 0.9438612466623206
computer_hardware SVR 0.5095475120050046
computer_hardware DT 0.8747537575581101
computer_hardware NN -11.976036098711017
parkinsons LR 0.9068547520909843
parkinsons L 0.8983382060322662
parkinsons R 0.9049789120180208


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


parkinsons SVR 0.8635415771581765
parkinsons DT 0.9986211277437788
parkinsons NN 0.973632082746802


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


여기까지가 classification, regression -> supervised learning 기계학습의 큰 두가지를 배웠음
기계학습 인공지능이다